# Can we monitor what7 redditors say recently?

https://github.com/pushshift/api/blob/master/README.md

https://github.com/dmarx/psaw

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from pmaw import PushshiftAPI

kw = 'GME'
subreddit='wallstreetbets'
obs_hrs = 12

curr_obs_end = dt.timestamp(dt.now().replace(minute=0, second=0, microsecond=0))
curr_obs_start = curr_obs_end - (obs_hrs * 3600)
prev_obs_end = curr_obs_end - 86400
prev_obs_start = curr_obs_start - 86400

api = PushshiftAPI()
gen = api.search_comments(q=kw, subreddit=subreddit, 
                          after=int(prev_obs_start), 
                          before=int(curr_obs_end))
df = pd.DataFrame([result for result in gen])

In [ ]:
df.loc[df['created_utc'].between(curr_obs_start, curr_obs_end), 'set'] = "Today"
df.loc[df['created_utc'].between(prev_obs_start, prev_obs_end), 'set'] = "Yesterday"
df = df[df['set'].notnull()]
df['created_utc'] = (df['set'] == "Yesterday") * 86400 + df['created_utc']

In [ ]:
df['created_ts'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)
df['created_ts'] = df['created_ts'].dt.tz_convert('Asia/Hong_Kong')

In [ ]:
import plotly.express as px

hist = px.histogram(df, x='created_ts', color='set', barmode="overlay", #nbins=obs_hrs,
                    title=f"Occurrence of {kw} in r/{subreddit} in the last {obs_hrs} hours")
hist.show()